In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
#Baseline
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif

# For matrics
from scipy.sparse import csr_matrix

# Factorization
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from catboost import CatBoostClassifier, Pool

# 1. Download Data

In [124]:
train = pd.read_csv('data_train.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('data_test.csv').drop('Unnamed: 0', axis=1)
feat = pd.read_csv('features.csv', sep='\t', header=0).drop('Unnamed: 0', axis=1)

# 2. Columns

In [125]:
USER_COL = 'id'
SERVICE_COL = 'vas_id'
ACTUAL_COL = 'bought_service'

# 3. IMPORTANT VARIABLE

#### 7.5% of Users in test are new user for us.


In [168]:
RANDOM_STATE = 21

# 4. Look at Data

In [126]:
train.head(5)

id  vas_id    buy_time  target
0   540968     8.0  1537131600     0.0
1  1454121     4.0  1531688400     0.0
2  2458816     1.0  1534107600     0.0
3  3535012     5.0  1535922000     0.0
4  1693214     1.0  1535922000     0.0

In [300]:
test[test['id'] == 3130519]

id  vas_id    buy_time
0  3130519     2.0  1548018000

In [128]:
feat.head(5)

id    buy_time          0           1           2           3  \
0  2013026  1531688400  18.910029   46.980888    4.969214   -1.386798   
1  2014722  1539550800  36.690029  152.400888  448.069214  563.833202   
2  2015199  1545598800 -67.019971  157.050888  -63.180786  178.103202   
3  2021765  1534107600   7.010029  150.200888   -6.930786  216.213202   
4  2027465  1533502800 -90.439971  134.220888 -104.380786  153.643202   

            4          5         6          7  ...          243          244  \
0    3.791754  -14.01179 -16.08618 -65.076097  ...  -977.373846  -613.770792   
1  463.841754  568.99821 -16.08618 -53.216097  ...  -891.373846  -544.770792   
2  -68.598246  156.99821   3.51382  25.183903  ...  -977.373846  -613.770792   
3   76.621754  351.84821 -16.08618 -65.076097  ...  -973.373846  -613.770792   
4 -109.798246  132.53821 -16.08618 -65.076097  ...  1643.626154  2007.229208   

          245        246           247        248       249        250  \
0  -25.996269 -37.630448   -301.747724 -25.832889 -0.694428 -12.175933   
1  -20.996269  48.369552     80.252276 -13.832889 -0.694428  -1.175933   
2  -12.996269 -37.630448  10829.252276 -25.832889 -0.694428 -12.175933   
3  -23.996269 -37.630448   -205.747724 -24.832889 -0.694428 -11.175933   
4  206.003731 -21.630448   6667.252276  92.167111 -0.694428  49.824067   

        251  252  
0  -0.45614  0.0  
1  -0.45614  0.0  
2  -0.45614  0.0  
3  -0.45614  1.0  
4  47.54386  0.0  

[5 rows x 255 columns]

# 4.1 Does all users from train in test

In [129]:
common_users = train[USER_COL].unique()

In [130]:
len(test) == len(test[test[USER_COL].isin(common_users)])

False

In [131]:
test[~test[USER_COL].isin(common_users)][USER_COL].nunique()

65964

In [132]:
labels = ['common_users', 'solo_test_user']
colors = ['yellow','green']
plt.title('Cold start users')
plt.pie([train[USER_COL].nunique(), test[~test[USER_COL].isin(common_users)][USER_COL].nunique()],
        labels=labels,
        colors=colors,
        startangle=180,
        autopct = "%0.2f%%",)
plt.figure(figsize=(8, 6), dpi=80)
plt.show()

<Figure size 640x480 with 0 Axes>

## 4.2 Target balance

In [133]:
train.target.value_counts()

0.0    771467
1.0     60186
Name: target, dtype: int64

In [134]:
plt.title('Our service reccomend - purchase history')
plt.bar(height=train.target.value_counts(), x=["Didn't buy", 'buy'])

<BarContainer object of 2 artists>

## 4.3 How much services did users from train buy?

In [135]:
train_user_service_df = train.groupby(USER_COL)[SERVICE_COL].unique().reset_index()
train_user_service_df.columns=[USER_COL, ACTUAL_COL]

In [136]:
train_user_service_df['bought_services'] = train_user_service_df[ACTUAL_COL].apply(lambda x: len(x))

In [137]:
train_user_service_df['bought_services'].unique()

array([1, 2, 3])

In [138]:
train_user_service_df['bought_services'].value_counts()

1    787847
2     18741
3        25
Name: bought_services, dtype: int64

## 4.4 How much services do we have at all?

In [139]:
train[SERVICE_COL].unique()

array([8., 4., 1., 5., 2., 7., 6., 9.])

In [140]:
train[SERVICE_COL].value_counts()

1.0    310175
2.0    249505
5.0     94085
4.0     85756
6.0     57878
7.0     15432
8.0     13350
9.0      5472
Name: vas_id, dtype: int64

In [4]:
train = pd.read_csv('data_train.csv').drop('Unnamed: 0', axis=1)

In [5]:
train.head(1)

id  vas_id    buy_time  target
0  540968     8.0  1537131600     0.0

In [6]:
o = train.groupby('vas_id')['target'].sum().reset_index()

In [7]:
o 

vas_id   target
0     1.0   5664.0
1     2.0   4797.0
2     4.0  21765.0
3     5.0   1692.0
4     6.0  24704.0
5     7.0    213.0
6     8.0    347.0
7     9.0   1004.0

In [141]:
len(train)

831653

In [142]:
train.buy_time.unique()

array([1537131600, 1531688400, 1534107600, 1535922000, 1532898000,
       1544994000, 1545598800, 1534712400, 1541970000, 1538341200,
       1533502800, 1541365200, 1539550800, 1536526800, 1546203600,
       1532293200, 1544389200, 1542574800, 1537736400, 1535317200,
       1540760400, 1531083600, 1538946000, 1540155600, 1543784400,
       1543179600])

In [143]:
len(feat['buy_time'])

4512528

In [144]:
len(feat.drop_duplicates())

4512528

# 5. Modify Train

In [150]:
train = train.merge(feat, on='id', how='left')

In [152]:
train[train.buy_time_x != train.buy_time_y]

id  vas_id  buy_time_x  target  buy_time_y           0  \
0        540968     8.0  1537131600     0.0  1541970000  -31.559971   
1       1454121     4.0  1531688400     0.0  1531083600  547.270029   
2       2458816     1.0  1534107600     0.0  1543179600  -92.139971   
3       3535012     5.0  1535922000     0.0  1533502800   54.880029   
4       1693214     1.0  1535922000     0.0  1543179600   45.160029   
...         ...     ...         ...     ...         ...         ...   
860047  3812226     2.0  1546203600     0.0  1532898000   29.750029   
860048  2480469     2.0  1546203600     0.0  1532293200  -89.179971   
860049   158236     2.0  1546203600     0.0  1538946000  -96.799971   
860050  1825525     2.0  1546203600     0.0  1533502800  -96.799971   
860051  3022610     2.0  1546203600     0.0  1540760400  -49.339971   

                 1           2           3           4  ...          243  \
0       327.360888  -45.500786  274.753202  -50.918246  ...  -845.373846   
1       238.430888  533.329214  274.803202  527.911754  ...  -972.373846   
2       -95.469112 -106.080786 -139.596798 -111.498246  ...  -977.373846   
3        12.970888   54.079214   -9.116798   48.661754  ...  -977.373846   
4       295.240888   64.679214  344.283202   59.261754  ...  -965.373846   
...            ...         ...         ...         ...  ...          ...   
860047    6.200888   24.279214  -37.936798   18.861754  ...  -977.373846   
860048  -56.499112 -103.120786 -109.106798 -108.538246  ...   -66.373846   
860049   62.140888 -110.740786    9.533202 -116.158246  ...  -683.373846   
860050  -81.919112 -110.740786 -128.596798 -116.158246  ...  1018.626154   
860051 -218.339112   73.139214  145.093202   67.721754  ...  -977.373846   

                244        245        246          247         248       249  \
0       -613.770792 -20.996269 -37.630448   -28.747724    4.167111  7.305572   
1       -613.770792 -25.996269 -19.630448  -278.747724  -24.832889 -0.694428   
2       -613.770792 -25.996269 -37.630448  -304.747724  -25.832889 -0.694428   
3       -613.770792 -25.996269 -18.630448  -133.747724  -14.832889 -0.694428   
4       -612.770792 -22.996269 -32.630448  -127.747724   -4.832889 -0.694428   
...             ...        ...        ...          ...         ...       ...   
860047  -613.770792 -25.996269 -37.630448  -306.747724  -25.832889 -0.694428   
860048  -321.770792 -25.996269  62.369552  1167.252276   41.167111 -0.694428   
860049  -470.770792 -25.996269 -37.630448    99.252276  178.167111 -0.694428   
860050  1367.229208 -24.996269 -35.630448  -237.747724  -21.832889 -0.694428   
860051  -613.770792 -25.996269 -37.630448  -303.747724  -25.832889 -0.694428   

               250       251  252  
0       -12.175933  21.54386  0.0  
1       -11.175933  -0.45614  0.0  
2       -12.175933  -0.45614  0.0  
3        -1.175933  -0.45614  0.0  
4       -12.175933  -0.45614  0.0  
...            ...       ...  ...  
860047  -12.175933  -0.45614  0.0  
860048   54.824067  -0.45614  0.0  
860049  191.824067  -0.45614  0.0  
860050   -8.175933  -0.45614  1.0  
860051  -12.175933  -0.45614  0.0  

[821133 rows x 258 columns]

In [155]:
train['time'] = train.buy_time_y - train.buy_time_x

In [165]:
train[train['time'] > 0]

id  vas_id  buy_time_x  target  buy_time_y          0  \
0        540968     8.0  1537131600     0.0  1541970000 -31.559971   
2       2458816     1.0  1534107600     0.0  1543179600 -92.139971   
4       1693214     1.0  1535922000     0.0  1543179600  45.160029   
5       3173665     8.0  1532898000     0.0  1537736400 -67.149971   
11      1387603     4.0  1535922000     0.0  1548018000 -95.299971   
...         ...     ...         ...     ...         ...        ...   
860029  3431643     5.0  1546203600     0.0  1547413200 -95.299971   
860034  1232859     2.0  1546203600     0.0  1547413200 -96.799971   
860038  3173649     2.0  1546203600     0.0  1547413200 -96.799971   
860040   742860     2.0  1546203600     0.0  1546808400 -96.799971   
860044    90279     2.0  1546203600     0.0  1547413200 -96.799971   

                  1           2            3           4  ...          244  \
0        327.360888  -45.500786   274.753202  -50.918246  ...  -613.770792   
2        -95.469112 -106.080786  -139.596798 -111.498246  ...  -613.770792   
4        295.240888   64.679214   344.283202   59.261754  ...  -612.770792   
5       -372.599112  -81.090786  -425.206798  -86.508246  ...  -613.770792   
11        94.820888 -102.570786   168.043202 -107.988246  ...   456.229208   
...             ...         ...          ...         ...  ...          ...   
860029  -248.349112 -109.240786  -300.956798 -114.658246  ...  -613.770792   
860034  -196.319112 -110.740786  -235.586798 -113.158246  ...  -613.770792   
860038   101.820888 -110.740786    49.213202 -116.158246  ...  5550.229208   
860040  2152.110888 -110.740786  2105.853202 -116.158246  ...  -598.770792   
860044  -208.179112 -101.430786  -224.666798 -106.848246  ...  2668.229208   

              245        246          247        248       249        250  \
0      -20.996269 -37.630448   -28.747724   4.167111  7.305572 -12.175933   
2      -25.996269 -37.630448  -304.747724 -25.832889 -0.694428 -12.175933   
4      -22.996269 -32.630448  -127.747724  -4.832889 -0.694428 -12.175933   
5      -25.996269 -37.630448  -306.747724 -25.832889 -0.694428 -12.175933   
11     -24.996269 -13.630448  -226.747724  34.167111 -0.694428 -12.175933   
...           ...        ...          ...        ...       ...        ...   
860029 -25.996269 -37.630448  -186.747724 -25.832889 -0.694428 -12.175933   
860034 -24.996269   4.369552  -202.747724  14.167111 -0.694428 -12.175933   
860038 -25.996269 -34.630448   149.252276 -11.832889 -0.694428 -12.175933   
860040 -25.996269 -37.630448  -300.747724 -25.832889 -0.694428 -12.175933   
860044 -17.996269  -4.630448  1854.252276  12.167111 -0.694428   8.824067   

             251  252      time  
0       21.54386  0.0   4838400  
2       -0.45614  0.0   9072000  
4       -0.45614  0.0   7257600  
5       -0.45614  0.0   4838400  
11      -0.45614  0.0  12096000  
...          ...  ...       ...  
860029  -0.45614  0.0   1209600  
860034  -0.45614  0.0   1209600  
860038  -0.45614  0.0   1209600  
860040  -0.45614  0.0    604800  
860044  -0.45614  0.0   1209600  

[443742 rows x 259 columns]

In [304]:
train.target.value_counts()

0.0    797845
1.0     62207
Name: target, dtype: int64

# 6. DROP AND CONVERT COLUMNS

In [166]:
train = train.drop(['buy_time_x', 'buy_time_y'], axis=1)

In [169]:
# pass

# 7. Train - Val Split

In [175]:
train_idx, test_idx = train_test_split(train.index, test_size=0.25, random_state=RANDOM_STATE)

y_train, y_test = train['target'].loc[train_idx], train['target'].loc[test_idx]
X_train, X_test = train.drop('target', axis=1).loc[train_idx], train.drop('target', axis=1).loc[test_idx]

print("Train")
print(train['target'].loc[train_idx].value_counts()/train_idx.shape[0])
print()
print("Test")
print(train['target'].loc[test_idx].value_counts()/test_idx.shape[0])

Train
0.0    0.92757
1.0    0.07243
Name: target, dtype: float64

Test
0.0    0.927972
1.0    0.072028
Name: target, dtype: float64


# 8.1 Baseline Score

# 10. Baseline Recomend With sklearn

In [298]:
clf = LogisticRegression()

In [305]:
clf.fit(X_train, y_train)

LogisticRegression()

In [307]:
y_pred = clf.predict(X_test)

In [309]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.98      0.95    199526
         1.0       0.13      0.04      0.07     15487

    accuracy                           0.91    215013
   macro avg       0.53      0.51      0.51    215013
weighted avg       0.87      0.91      0.89    215013



In [312]:
f1_score(y_test, y_pred, average='macro')

0.5095677535306835

In [314]:
result = result.append({'model': 'LogReg', 'f1_macro': f1_score(y_test, y_pred, average='macro')}, ignore_index=True)

In [315]:
result

model  f1_macro
0    random_all  0.094883
1  random_top_3  0.094883
2        LogReg  0.509568

# Решил Порисовать

# Катбуст

In [321]:
model = CatBoostClassifier(iterations=1000,
                           depth=10, 
                           l2_leaf_reg=5,
                           loss_function='Logloss', #Logloss
                           learning_rate=0.1,
                          )

In [322]:
model.fit(X_train, y_train)

0:	learn: 0.5502672	total: 798ms	remaining: 13m 16s
1:	learn: 0.4480988	total: 1.35s	remaining: 11m 16s
2:	learn: 0.3757299	total: 1.92s	remaining: 10m 38s
3:	learn: 0.3348584	total: 2.52s	remaining: 10m 26s
4:	learn: 0.2931559	total: 3.05s	remaining: 10m 6s
5:	learn: 0.2718748	total: 3.62s	remaining: 10m
6:	learn: 0.2515402	total: 4.12s	remaining: 9m 43s
7:	learn: 0.2389192	total: 4.69s	remaining: 9m 42s
8:	learn: 0.2296590	total: 5.25s	remaining: 9m 37s
9:	learn: 0.2173342	total: 5.72s	remaining: 9m 26s
10:	learn: 0.2086813	total: 6.22s	remaining: 9m 19s
11:	learn: 0.2023477	total: 6.69s	remaining: 9m 11s
12:	learn: 0.1984263	total: 7.25s	remaining: 9m 10s
13:	learn: 0.1955025	total: 7.83s	remaining: 9m 11s
14:	learn: 0.1930769	total: 8.4s	remaining: 9m 11s
15:	learn: 0.1921053	total: 8.95s	remaining: 9m 10s
16:	learn: 0.1891166	total: 9.47s	remaining: 9m 7s
17:	learn: 0.1882316	total: 10s	remaining: 9m 7s
18:	learn: 0.1860064	total: 10.5s	remaining: 9m 3s
19:	learn: 0.1850664	total:

158:	learn: 0.1664309	total: 1m 25s	remaining: 7m 32s
159:	learn: 0.1663833	total: 1m 26s	remaining: 7m 32s
160:	learn: 0.1663197	total: 1m 26s	remaining: 7m 31s
161:	learn: 0.1663152	total: 1m 27s	remaining: 7m 31s
162:	learn: 0.1662984	total: 1m 27s	remaining: 7m 30s
163:	learn: 0.1662874	total: 1m 28s	remaining: 7m 30s
164:	learn: 0.1662438	total: 1m 28s	remaining: 7m 29s
165:	learn: 0.1662193	total: 1m 29s	remaining: 7m 29s
166:	learn: 0.1661917	total: 1m 30s	remaining: 7m 29s
167:	learn: 0.1661742	total: 1m 30s	remaining: 7m 28s
168:	learn: 0.1661690	total: 1m 31s	remaining: 7m 28s
169:	learn: 0.1660976	total: 1m 31s	remaining: 7m 28s
170:	learn: 0.1660056	total: 1m 32s	remaining: 7m 27s
171:	learn: 0.1659652	total: 1m 32s	remaining: 7m 26s
172:	learn: 0.1658993	total: 1m 33s	remaining: 7m 26s
173:	learn: 0.1658729	total: 1m 33s	remaining: 7m 25s
174:	learn: 0.1658408	total: 1m 34s	remaining: 7m 25s
175:	learn: 0.1658103	total: 1m 35s	remaining: 7m 25s
176:	learn: 0.1658077	total:

311:	learn: 0.1591841	total: 2m 49s	remaining: 6m 14s
312:	learn: 0.1591284	total: 2m 50s	remaining: 6m 13s
313:	learn: 0.1590836	total: 2m 50s	remaining: 6m 13s
314:	learn: 0.1590766	total: 2m 51s	remaining: 6m 12s
315:	learn: 0.1589819	total: 2m 52s	remaining: 6m 12s
316:	learn: 0.1589365	total: 2m 52s	remaining: 6m 11s
317:	learn: 0.1588894	total: 2m 53s	remaining: 6m 11s
318:	learn: 0.1588238	total: 2m 53s	remaining: 6m 10s
319:	learn: 0.1587806	total: 2m 54s	remaining: 6m 10s
320:	learn: 0.1587426	total: 2m 54s	remaining: 6m 9s
321:	learn: 0.1587032	total: 2m 55s	remaining: 6m 9s
322:	learn: 0.1586716	total: 2m 55s	remaining: 6m 8s
323:	learn: 0.1586299	total: 2m 56s	remaining: 6m 8s
324:	learn: 0.1585118	total: 2m 57s	remaining: 6m 7s
325:	learn: 0.1584167	total: 2m 57s	remaining: 6m 7s
326:	learn: 0.1583782	total: 2m 58s	remaining: 6m 6s
327:	learn: 0.1583180	total: 2m 58s	remaining: 6m 6s
328:	learn: 0.1582862	total: 2m 59s	remaining: 6m 5s
329:	learn: 0.1582729	total: 2m 59s	r

465:	learn: 0.1521766	total: 4m 14s	remaining: 4m 51s
466:	learn: 0.1521335	total: 4m 15s	remaining: 4m 51s
467:	learn: 0.1520890	total: 4m 15s	remaining: 4m 50s
468:	learn: 0.1520867	total: 4m 16s	remaining: 4m 49s
469:	learn: 0.1520359	total: 4m 16s	remaining: 4m 49s
470:	learn: 0.1519602	total: 4m 17s	remaining: 4m 48s
471:	learn: 0.1519081	total: 4m 17s	remaining: 4m 48s
472:	learn: 0.1518350	total: 4m 18s	remaining: 4m 47s
473:	learn: 0.1517914	total: 4m 18s	remaining: 4m 47s
474:	learn: 0.1517613	total: 4m 19s	remaining: 4m 46s
475:	learn: 0.1516945	total: 4m 19s	remaining: 4m 46s
476:	learn: 0.1516259	total: 4m 20s	remaining: 4m 45s
477:	learn: 0.1516154	total: 4m 20s	remaining: 4m 45s
478:	learn: 0.1515540	total: 4m 21s	remaining: 4m 44s
479:	learn: 0.1514808	total: 4m 22s	remaining: 4m 43s
480:	learn: 0.1514518	total: 4m 22s	remaining: 4m 43s
481:	learn: 0.1513826	total: 4m 23s	remaining: 4m 42s
482:	learn: 0.1513286	total: 4m 23s	remaining: 4m 42s
483:	learn: 0.1513052	total:

618:	learn: 0.1457673	total: 5m 39s	remaining: 3m 28s
619:	learn: 0.1457620	total: 5m 40s	remaining: 3m 28s
620:	learn: 0.1457063	total: 5m 40s	remaining: 3m 27s
621:	learn: 0.1457044	total: 5m 41s	remaining: 3m 27s
622:	learn: 0.1456650	total: 5m 41s	remaining: 3m 26s
623:	learn: 0.1456461	total: 5m 42s	remaining: 3m 26s
624:	learn: 0.1456430	total: 5m 42s	remaining: 3m 25s
625:	learn: 0.1456132	total: 5m 43s	remaining: 3m 25s
626:	learn: 0.1455706	total: 5m 43s	remaining: 3m 24s
627:	learn: 0.1455385	total: 5m 44s	remaining: 3m 24s
628:	learn: 0.1454872	total: 5m 45s	remaining: 3m 23s
629:	learn: 0.1454850	total: 5m 45s	remaining: 3m 23s
630:	learn: 0.1454821	total: 5m 46s	remaining: 3m 22s
631:	learn: 0.1454189	total: 5m 46s	remaining: 3m 21s
632:	learn: 0.1453469	total: 5m 47s	remaining: 3m 21s
633:	learn: 0.1453099	total: 5m 47s	remaining: 3m 20s
634:	learn: 0.1453087	total: 5m 48s	remaining: 3m 20s
635:	learn: 0.1452606	total: 5m 48s	remaining: 3m 19s
636:	learn: 0.1452532	total:

771:	learn: 0.1401650	total: 7m 3s	remaining: 2m 5s
772:	learn: 0.1401286	total: 7m 3s	remaining: 2m 4s
773:	learn: 0.1401043	total: 7m 4s	remaining: 2m 3s
774:	learn: 0.1400977	total: 7m 5s	remaining: 2m 3s
775:	learn: 0.1400970	total: 7m 5s	remaining: 2m 2s
776:	learn: 0.1400657	total: 7m 6s	remaining: 2m 2s
777:	learn: 0.1400364	total: 7m 6s	remaining: 2m 1s
778:	learn: 0.1399593	total: 7m 7s	remaining: 2m 1s
779:	learn: 0.1398946	total: 7m 7s	remaining: 2m
780:	learn: 0.1398747	total: 7m 8s	remaining: 2m
781:	learn: 0.1398491	total: 7m 8s	remaining: 1m 59s
782:	learn: 0.1398119	total: 7m 9s	remaining: 1m 58s
783:	learn: 0.1397603	total: 7m 9s	remaining: 1m 58s
784:	learn: 0.1397361	total: 7m 10s	remaining: 1m 57s
785:	learn: 0.1397212	total: 7m 10s	remaining: 1m 57s
786:	learn: 0.1396819	total: 7m 11s	remaining: 1m 56s
787:	learn: 0.1396260	total: 7m 12s	remaining: 1m 56s
788:	learn: 0.1395829	total: 7m 12s	remaining: 1m 55s
789:	learn: 0.1395232	total: 7m 13s	remaining: 1m 55s
790

925:	learn: 0.1337415	total: 8m 27s	remaining: 40.6s
926:	learn: 0.1336784	total: 8m 27s	remaining: 40s
927:	learn: 0.1336136	total: 8m 28s	remaining: 39.5s
928:	learn: 0.1335551	total: 8m 29s	remaining: 38.9s
929:	learn: 0.1335352	total: 8m 29s	remaining: 38.4s
930:	learn: 0.1334810	total: 8m 30s	remaining: 37.8s
931:	learn: 0.1334512	total: 8m 30s	remaining: 37.3s
932:	learn: 0.1333990	total: 8m 31s	remaining: 36.7s
933:	learn: 0.1333771	total: 8m 31s	remaining: 36.2s
934:	learn: 0.1333215	total: 8m 32s	remaining: 35.6s
935:	learn: 0.1332524	total: 8m 32s	remaining: 35.1s
936:	learn: 0.1332309	total: 8m 33s	remaining: 34.5s
937:	learn: 0.1331958	total: 8m 33s	remaining: 34s
938:	learn: 0.1331496	total: 8m 34s	remaining: 33.4s
939:	learn: 0.1330851	total: 8m 34s	remaining: 32.9s
940:	learn: 0.1330500	total: 8m 35s	remaining: 32.3s
941:	learn: 0.1330138	total: 8m 36s	remaining: 31.8s
942:	learn: 0.1329829	total: 8m 36s	remaining: 31.2s
943:	learn: 0.1329475	total: 8m 37s	remaining: 30.

In [323]:
y_pred = model.predict(X_test)

In [324]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96    199526
         1.0       0.50      0.15      0.23     15487

    accuracy                           0.93    215013
   macro avg       0.72      0.57      0.60    215013
weighted avg       0.91      0.93      0.91    215013



In [325]:
f1_score(y_test, y_pred, average='macro')

0.5973039945667431

# Вот и получили первые результаты пошаманим с признаками

In [327]:
corr_matrix = train.corr().abs()

In [330]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

<ipython-input-330-c1ad37c011f5>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [331]:
upper

id    vas_id    target         0         1         2         3  \
id     NaN  0.015279  0.061817  0.018473  0.049970  0.018921  0.047408   
vas_id NaN       NaN  0.262709  0.000181  0.001011  0.000360  0.000808   
target NaN       NaN       NaN  0.001340  0.003513  0.001632  0.003634   
0      NaN       NaN       NaN       NaN  0.609153  0.987123  0.554705   
1      NaN       NaN       NaN       NaN       NaN  0.612404  0.920726   
...     ..       ...       ...       ...       ...       ...       ...   
249    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
250    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
251    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
252    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
time   NaN       NaN       NaN       NaN       NaN       NaN       NaN   

               4         5         6  ...       244       245       246  \
id      0.020839  0.048798  0.005398  ...  0.032342  0.002065  0.018201   
vas_id  0.000358  0.000989  0.000052  ...  0.001114  0.001517  0.001754   
target  0.002008  0.003730  0.000604  ...  0.001324  0.003266  0.000206   
0       0.963982  0.522765  0.839958  ...  0.023275  0.002822  0.013776   
1       0.612693  0.884849  0.547745  ...  0.067072  0.009240  0.036229   
...          ...       ...       ...  ...       ...       ...       ...   
249          NaN       NaN       NaN  ...       NaN       NaN       NaN   
250          NaN       NaN       NaN  ...       NaN       NaN       NaN   
251          NaN       NaN       NaN  ...       NaN       NaN       NaN   
252          NaN       NaN       NaN  ...       NaN       NaN       NaN   
time         NaN       NaN       NaN  ...       NaN       NaN       NaN   

             247       248       249       250       251       252      time  
id      0.017423  0.007301  0.002975  0.000932  0.002292  0.011908  0.063104  
vas_id  0.000243  0.002671  0.000518  0.002397  0.000790  0.000133  0.174991  
target  0.001534  0.001410  0.000487  0.000622  0.000511  0.000513  0.101398  
0       0.025549  0.006708  0.012711  0.004651  0.006533  0.009406  0.016402  
1       0.072561  0.026444  0.034112  0.016886  0.016072  0.022774  0.040471  
...          ...       ...       ...       ...       ...       ...       ...  
249          NaN       NaN       NaN  0.007310  0.016117  0.002201  0.000067  
250          NaN       NaN       NaN       NaN  0.006775  0.000314  0.035537  
251          NaN       NaN       NaN       NaN       NaN  0.000836  0.002309  
252          NaN       NaN       NaN       NaN       NaN       NaN  0.005104  
time         NaN       NaN       NaN       NaN       NaN       NaN       NaN  

[257 rows x 257 columns]

In [334]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [335]:
to_drop

['2',
 '4',
 '5',
 '14',
 '33',
 '35',
 '51',
 '71',
 '72',
 '78',
 '79',
 '112',
 '113',
 '116',
 '123',
 '124',
 '137',
 '138',
 '142',
 '151',
 '162',
 '170',
 '186',
 '217',
 '220']

In [336]:
train = train.drop(to_drop, axis=1)

# Удаление не релевантных признаков

In [365]:
fvalue_selector = SelectKBest(f_classif, k=50)

In [362]:
target = train.target
data = train.drop('target', axis=1).abs().astype(int)

In [366]:
f_kbest = fvalue_selector.fit_transform(data, target)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 24  25  29  30  52  55  56  57  58  59  68  72  76 103 104 112 113 114
 118 119 123 131 132 133 141 142 143 145 171 173 174 175 176 177 178 179
 180 181 182 183 184 185] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,


In [367]:
data.shape

(860052, 231)

# Протестируем?

In [372]:
train_1 = train.copy()

In [384]:
train_1 = train_1[train_1.columns[:3]]

In [387]:
f_kbest = pd.DataFrame(f_kbest)

In [391]:
for i in range(50):
    train_1[i] = f_kbest[i]

<ipython-input-391-e10227dd05dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1[i] = f_kbest[i]


In [392]:
train_1

id  vas_id  target        0  1    2  3    4   5  6  ...   40  \
0        540968     8.0     0.0   540968  8  274  5   13  54  0  ...  270   
1       1454121     4.0     0.0  1454121  4  274  5   13  10  0  ...   12   
2       2458816     1.0     0.0  2458816  1  139  5   13  10  0  ...   39   
3       3535012     5.0     0.0  3535012  5    9  5    0  10  0  ...   38   
4       1693214     1.0     0.0  1693214  1  344  5   19  10  0  ...    9   
...         ...     ...     ...      ... ..  ... ..  ...  .. ..  ...  ...   
860047  3812226     2.0     0.0  3812226  2   37  5    5  10  0  ...   39   
860048  2480469     2.0     0.0  2480469  2  109  5   13  10  0  ...   21   
860049   158236     2.0     0.0   158236  2    9  5   13  10  0  ...   39   
860050  1825525     2.0     0.0  1825525  2  128  5   13  10  0  ...   38   
860051  3022610     2.0     0.0  3022610  2  145  5  122  10  0  ...   39   

         41   42    43   44   45   46  47    48        49  
0       269  109   364  938  504   53  20    28   4838400  
1         8  109   876  247  512   80  25   278    604800  
2        35  109   703  247  339  106  25   304   9072000  
3        35  109   574  244  324   90  25   133   2419200  
4        11  109    52  269  471   80  22   127   7257600  
...     ...  ...   ...  ...  ...  ...  ..   ...       ...  
860047   35  109   876  247  512  106  25   306  13305600  
860048   26  109   487  898  322  955  25  1167  13910400  
860049   35  109  1912  247  501   18  25    99   7257600  
860050   34  109   451   11  348  105  24   237  12700800  
860051   35  109   779  246  416  106  25   303   5443200  

[860052 rows x 53 columns]

# Train-Test

In [399]:
train_idx, test_idx = train_test_split(train.index, test_size=0.25, random_state=RANDOM_STATE)

y_train, y_test = train['target'].loc[train_idx], train['target'].loc[test_idx]
X_train, X_test = train.drop('target', axis=1).loc[train_idx], train.drop('target', axis=1).loc[test_idx]

print("Train")
print(train['target'].loc[train_idx].value_counts()/train_idx.shape[0])
print()
print("Test")
print(train['target'].loc[test_idx].value_counts()/test_idx.shape[0])

Train
0.0    0.92757
1.0    0.07243
Name: target, dtype: float64

Test
0.0    0.927972
1.0    0.072028
Name: target, dtype: float64


In [400]:
model.fit(X_train, y_train)

0:	learn: 0.5497749	total: 499ms	remaining: 8m 18s
1:	learn: 0.4477515	total: 990ms	remaining: 8m 13s
2:	learn: 0.3796196	total: 1.51s	remaining: 8m 21s
3:	learn: 0.3381062	total: 2.01s	remaining: 8m 21s
4:	learn: 0.3070700	total: 2.55s	remaining: 8m 26s
5:	learn: 0.2738442	total: 3.04s	remaining: 8m 24s
6:	learn: 0.2502936	total: 3.5s	remaining: 8m 17s
7:	learn: 0.2349613	total: 3.94s	remaining: 8m 7s
8:	learn: 0.2222875	total: 4.39s	remaining: 8m 3s
9:	learn: 0.2173506	total: 4.87s	remaining: 8m 2s
10:	learn: 0.2114466	total: 5.38s	remaining: 8m 4s
11:	learn: 0.2040764	total: 5.8s	remaining: 7m 57s
12:	learn: 0.1986965	total: 6.22s	remaining: 7m 51s
13:	learn: 0.1945546	total: 6.64s	remaining: 7m 47s
14:	learn: 0.1919513	total: 7.13s	remaining: 7m 48s
15:	learn: 0.1892935	total: 7.62s	remaining: 7m 48s
16:	learn: 0.1879716	total: 8.1s	remaining: 7m 48s
17:	learn: 0.1862549	total: 8.52s	remaining: 7m 44s
18:	learn: 0.1851823	total: 9.03s	remaining: 7m 46s
19:	learn: 0.1839977	total: 9

158:	learn: 0.1665660	total: 1m 16s	remaining: 6m 43s
159:	learn: 0.1665316	total: 1m 16s	remaining: 6m 43s
160:	learn: 0.1664070	total: 1m 17s	remaining: 6m 42s
161:	learn: 0.1663679	total: 1m 17s	remaining: 6m 42s
162:	learn: 0.1663033	total: 1m 18s	remaining: 6m 41s
163:	learn: 0.1661154	total: 1m 18s	remaining: 6m 41s
164:	learn: 0.1660689	total: 1m 19s	remaining: 6m 40s
165:	learn: 0.1659411	total: 1m 19s	remaining: 6m 40s
166:	learn: 0.1658142	total: 1m 20s	remaining: 6m 39s
167:	learn: 0.1657935	total: 1m 20s	remaining: 6m 39s
168:	learn: 0.1657393	total: 1m 21s	remaining: 6m 39s
169:	learn: 0.1655904	total: 1m 21s	remaining: 6m 38s
170:	learn: 0.1655778	total: 1m 22s	remaining: 6m 38s
171:	learn: 0.1655488	total: 1m 22s	remaining: 6m 38s
172:	learn: 0.1654850	total: 1m 23s	remaining: 6m 37s
173:	learn: 0.1654010	total: 1m 23s	remaining: 6m 37s
174:	learn: 0.1653006	total: 1m 24s	remaining: 6m 36s
175:	learn: 0.1652054	total: 1m 24s	remaining: 6m 35s
176:	learn: 0.1651445	total:

311:	learn: 0.1582881	total: 2m 31s	remaining: 5m 33s
312:	learn: 0.1582497	total: 2m 31s	remaining: 5m 33s
313:	learn: 0.1582276	total: 2m 32s	remaining: 5m 32s
314:	learn: 0.1582269	total: 2m 32s	remaining: 5m 32s
315:	learn: 0.1582125	total: 2m 33s	remaining: 5m 31s
316:	learn: 0.1582110	total: 2m 33s	remaining: 5m 31s
317:	learn: 0.1581564	total: 2m 34s	remaining: 5m 31s
318:	learn: 0.1581285	total: 2m 34s	remaining: 5m 30s
319:	learn: 0.1580306	total: 2m 35s	remaining: 5m 30s
320:	learn: 0.1580301	total: 2m 35s	remaining: 5m 29s
321:	learn: 0.1579412	total: 2m 36s	remaining: 5m 29s
322:	learn: 0.1579039	total: 2m 36s	remaining: 5m 28s
323:	learn: 0.1578583	total: 2m 37s	remaining: 5m 28s
324:	learn: 0.1577943	total: 2m 37s	remaining: 5m 27s
325:	learn: 0.1577472	total: 2m 38s	remaining: 5m 27s
326:	learn: 0.1577214	total: 2m 38s	remaining: 5m 26s
327:	learn: 0.1577059	total: 2m 39s	remaining: 5m 26s
328:	learn: 0.1576627	total: 2m 39s	remaining: 5m 25s
329:	learn: 0.1576288	total:

464:	learn: 0.1519946	total: 3m 47s	remaining: 4m 21s
465:	learn: 0.1519103	total: 3m 47s	remaining: 4m 20s
466:	learn: 0.1518421	total: 3m 48s	remaining: 4m 20s
467:	learn: 0.1518064	total: 3m 48s	remaining: 4m 19s
468:	learn: 0.1518010	total: 3m 49s	remaining: 4m 19s
469:	learn: 0.1517921	total: 3m 49s	remaining: 4m 18s
470:	learn: 0.1517517	total: 3m 50s	remaining: 4m 18s
471:	learn: 0.1516609	total: 3m 50s	remaining: 4m 17s
472:	learn: 0.1516101	total: 3m 51s	remaining: 4m 17s
473:	learn: 0.1515458	total: 3m 51s	remaining: 4m 16s
474:	learn: 0.1515270	total: 3m 52s	remaining: 4m 16s
475:	learn: 0.1514772	total: 3m 52s	remaining: 4m 15s
476:	learn: 0.1514196	total: 3m 52s	remaining: 4m 15s
477:	learn: 0.1513835	total: 3m 53s	remaining: 4m 14s
478:	learn: 0.1513829	total: 3m 53s	remaining: 4m 14s
479:	learn: 0.1513308	total: 3m 54s	remaining: 4m 13s
480:	learn: 0.1513186	total: 3m 55s	remaining: 4m 13s
481:	learn: 0.1513009	total: 3m 55s	remaining: 4m 13s
482:	learn: 0.1512381	total:

617:	learn: 0.1457012	total: 5m 3s	remaining: 3m 7s
618:	learn: 0.1456528	total: 5m 3s	remaining: 3m 6s
619:	learn: 0.1456374	total: 5m 4s	remaining: 3m 6s
620:	learn: 0.1455646	total: 5m 4s	remaining: 3m 5s
621:	learn: 0.1455641	total: 5m 5s	remaining: 3m 5s
622:	learn: 0.1455633	total: 5m 5s	remaining: 3m 5s
623:	learn: 0.1455017	total: 5m 6s	remaining: 3m 4s
624:	learn: 0.1454682	total: 5m 6s	remaining: 3m 4s
625:	learn: 0.1454180	total: 5m 7s	remaining: 3m 3s
626:	learn: 0.1453678	total: 5m 7s	remaining: 3m 3s
627:	learn: 0.1453218	total: 5m 8s	remaining: 3m 2s
628:	learn: 0.1453098	total: 5m 8s	remaining: 3m 2s
629:	learn: 0.1452519	total: 5m 9s	remaining: 3m 1s
630:	learn: 0.1452110	total: 5m 9s	remaining: 3m 1s
631:	learn: 0.1451802	total: 5m 10s	remaining: 3m
632:	learn: 0.1451430	total: 5m 10s	remaining: 3m
633:	learn: 0.1451381	total: 5m 11s	remaining: 2m 59s
634:	learn: 0.1451085	total: 5m 11s	remaining: 2m 59s
635:	learn: 0.1450948	total: 5m 12s	remaining: 2m 58s
636:	learn

771:	learn: 0.1395514	total: 6m 19s	remaining: 1m 51s
772:	learn: 0.1395145	total: 6m 19s	remaining: 1m 51s
773:	learn: 0.1394763	total: 6m 20s	remaining: 1m 51s
774:	learn: 0.1393961	total: 6m 20s	remaining: 1m 50s
775:	learn: 0.1393541	total: 6m 21s	remaining: 1m 50s
776:	learn: 0.1393317	total: 6m 21s	remaining: 1m 49s
777:	learn: 0.1392486	total: 6m 22s	remaining: 1m 49s
778:	learn: 0.1392123	total: 6m 22s	remaining: 1m 48s
779:	learn: 0.1391431	total: 6m 23s	remaining: 1m 48s
780:	learn: 0.1390872	total: 6m 23s	remaining: 1m 47s
781:	learn: 0.1390372	total: 6m 23s	remaining: 1m 47s
782:	learn: 0.1389793	total: 6m 24s	remaining: 1m 46s
783:	learn: 0.1389047	total: 6m 24s	remaining: 1m 46s
784:	learn: 0.1388518	total: 6m 25s	remaining: 1m 45s
785:	learn: 0.1388382	total: 6m 25s	remaining: 1m 45s
786:	learn: 0.1388268	total: 6m 26s	remaining: 1m 44s
787:	learn: 0.1387715	total: 6m 26s	remaining: 1m 44s
788:	learn: 0.1387251	total: 6m 27s	remaining: 1m 43s
789:	learn: 0.1386552	total:

925:	learn: 0.1331050	total: 7m 34s	remaining: 36.3s
926:	learn: 0.1330965	total: 7m 34s	remaining: 35.8s
927:	learn: 0.1330931	total: 7m 35s	remaining: 35.3s
928:	learn: 0.1330706	total: 7m 35s	remaining: 34.8s
929:	learn: 0.1330703	total: 7m 36s	remaining: 34.3s
930:	learn: 0.1330533	total: 7m 36s	remaining: 33.8s
931:	learn: 0.1330046	total: 7m 37s	remaining: 33.4s
932:	learn: 0.1329650	total: 7m 37s	remaining: 32.9s
933:	learn: 0.1329491	total: 7m 38s	remaining: 32.4s
934:	learn: 0.1329196	total: 7m 38s	remaining: 31.9s
935:	learn: 0.1328860	total: 7m 39s	remaining: 31.4s
936:	learn: 0.1328575	total: 7m 39s	remaining: 30.9s
937:	learn: 0.1327850	total: 7m 40s	remaining: 30.4s
938:	learn: 0.1327563	total: 7m 40s	remaining: 29.9s
939:	learn: 0.1326562	total: 7m 41s	remaining: 29.4s
940:	learn: 0.1326289	total: 7m 41s	remaining: 28.9s
941:	learn: 0.1325749	total: 7m 42s	remaining: 28.4s
942:	learn: 0.1325478	total: 7m 42s	remaining: 28s
943:	learn: 0.1325050	total: 7m 43s	remaining: 2

In [401]:
y_pred = model.predict(X_test)

In [402]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96    199526
         1.0       0.51      0.15      0.23     15487

    accuracy                           0.93    215013
   macro avg       0.72      0.57      0.60    215013
weighted avg       0.91      0.93      0.91    215013



In [403]:
f1_score(y_test, y_pred, average='macro')

0.5972384716349545